### Import libraries

In [5]:
# Load Libraries 
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import socket

### Customize

In [6]:
my_input_file='0405 my_input.csv'
my_features = ['Utilization', 'Downtime', 'Energy_Consumption', 'Defect_Rate']
my_target='Failure_Risk'
my_host='127.0.0.7'

### Read data and run regression

In [7]:
# Read data
data = pd.read_csv(my_input_file)

# Train Regression Model for Failure Risk Prediction 
X = data[my_features]
y = data[my_target]

model = LinearRegression()
model.fit(X, y)

data['Predicted_Risk'] = model.predict(X)

### Build dashboard 

In [8]:
#Display Dash server address
ip_address = socket.gethostbyname(socket.gethostname())
print(f"Dash app running at http://{ip_address}:8050")

#  Build Interactive Dashboard with Dash 
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Manufacturing Dashboard with ML Predictions"),
    
    dcc.Dropdown(
        id='kpi_dropdown',
        options=[{'label': col, 'value': col} for col in my_features],
        value='Utilization',
        clearable=False
    ),

    dcc.Graph(id='kpi_graph'),

    html.Br(),
    html.H3("Predicted Equipment Failure Risk (Regression)"),
    dcc.Graph(
        figure=px.bar(
            data,
            x='Machine_ID',
            y='Predicted_Risk',
            color='Predicted_Risk',
            color_continuous_scale='Reds',
            title="ML-Predicted Failure Risk by Machine"
        )
    )
])

@app.callback(
    Output('kpi_graph', 'figure'),
    Input('kpi_dropdown', 'value')
)
def update_kpi_chart(kpi_selected):
    fig = px.bar(data, x='Machine_ID', y=kpi_selected, color='Predicted_Risk',
                 color_continuous_scale='Viridis',
                 title=f'{kpi_selected} by Machine (ML Risk Overlay)')
    return fig

if __name__ == '__main__':

    # For external access prefix the first option  with # and remove the # from the second option
    app.run(my_host, port=8050, debug=True)
    #app.run(host='0.0.0.0', port=8050, mode='external', debug=True)
    

Dash app running at http://192.168.1.73:8050
